# Preference Tuning for Summarization using Synthetic Data

**⏱️ Time to complete**: \<TODO\>

Preference tuning is a powerful tool that can optimize LLMs towards complex preferences that can not easily captured through supervised fine-tuning. However, manually annotating preferences between model outputs using human raters can be extremely time-consuming and expensive. Instead, synthetic preference data can be generated by scoring responses with large foundation models, allowing for much cheaper and scalable data collection!

Here we'll go through an end-to-end example for preference tuning of an open-source language model with synthetic data, covering data preprocessing, fine-tuning and evaluation. 

We will focus on the task of summarization for the [CNN/DailyMail](https://huggingface.co/datasets/abisee/cnn_dailymail) dataset. 

# Table of Contents
1. [Data Preprocessing](#step-1-data-preprocessing): In this section we cover how we can prepare preference data for the summarization task using an LLM-as-a-judge. 
2. [DPO Finetuning](#step-2-fine-tuning): This section will cover how you can fine-tune an open source model on the preference data on the Anyscale platform.
3. [Evaluation](#step-3-evaluation): The section will lay down a blue-print for evaluation and compare performance to that of closed source models like OpenAI's GPT-4.
4. [Iterative-DPO](#step-4-iterative): An optional step to further boost performance with iterative preference-tuning. 

First, let's make the necessary imports

In [2]:
import os
import yaml
import datasets
import openai

import ray.data

import pprint

# Step 1: Synthetic Data Generation

First, let's inspect the training dataset and look at an example. 

In [ ]:
hf_ds = datasets.load_dataset("abisee/cnn_dailymail", '3.0.0', split="train").shuffle(seed=21)
# extract a subset of 20000 articles
hf_ds_subset =  hf_ds.select(range(20000))

ray_ds = ray.data.from_huggingface(hf_ds_subset)
raw_example = ray_ds.take(1)[0]

In [3]:
pprint.pprint(raw_example, width=100)

{'article': 'Scam: Lisa Harrison, 34, promised customers low currency rates on US dollars and '
            'special deals . A wedding planner who stole £80,000 from couples in a bid to satisfy '
            "an 'out-of-control' online gambling addiction has been jailed. Lisa Harrison, 34, "
            'began taking money from her clients in summer 2013 by enticing them with low currency '
            'rates on US dollars and flight upgrades. She took money from 19 couples who had '
            'entrusted their savings to her after being promised the wedding of their dreams. It '
            'is understood that the company she worked for, iPlan New York, specialised in '
            'weddings in New York City. Her website iplannewyork.com, which has been taken down, '
            "said: 'iPlan New York was set up to create and style the perfect tailor made wedding "
            "for couples travelling to New York to get married! 'We are passionate about what we "
            'do and p

Now, we need to get preference data for pairs of summaries generated from the same article. Traditionally, this would involve generating summaries using the base model you wish to fine-tune and asking human annotators to provide a rating for each sample. In this example, we will employ a synthetic summary scoring method that scores the accuracy of a summary using the following metrics:

**Summary Scoring Metrics**
1. Multiple choice Q&A accuracy:
    - Given the original text, we use an LLM judge to generate 5 multiple choice questions about the text.
    - We then ask the LLM judge to answer the questions using only the summary, and record the number of questions correctly answered.
2. Word count: We simply count the number of words in the summary.

This allows us to construct a simple preference function between two summaries:

**Preference Function**
1. If both summary responses attain ≥3 multiple choice questions correct, we will prefer the shorter response. We do not care about Q&A accuracy beyond 3 correct answers, since the summary should not contain all information from the text.
2. Otherwise, we select the response that leads to more correctly answered multiple choice questions.

To generate the training data, we will generate 10 summaries from each article using the model we wish to fine-tune. Then, we will randomly sample pairs of summaries and use our preference function to annotate the preference between them.

For this example, we will use `Mistral-7B-Instruct-v0.1` as the base model to fine-tune and `Llama-3.1-70B-Instruct` as a judge.

Combining all this together, our data pre-processing pipeline is going to look as follows: 

![preprocessing](./assets/preprocessing.png?1)

# TODO: Instructions for pre-processing
\<Provide a better descrption for the data preprocessing and the choices made.\>

\<We have the relevant preprocessing code in `utils/generate_questions.py` and `utils/generate_summaries_and_scores.py`. You can run data generation as an Anyscale job with configs/generate_questions_job.yaml and configs/generate_summaries_job.yaml.\>

\<After preprocessing, here's an example for the Q&A generated by Llama 70B and here's an example for the summaries generated by Mistral 7B Instruct \>


\<We sample chosen and rejected messages from the summaries based on the Q&A Accuracy score. We use a threshold of 3/5 for classifying examples as 'chosen' and 'rejected'. Here's an example training dataset sample for the DPO model\>

### Generate Multiple Choice Questions from Articles

First, we will generate the multiple choice questions and answers for each article using `Llama-3.1-70B-Instruct`. Leveraging vLLM and Ray, we can very easily scale this generation process across multiple GPUs.

The following command will run the `TODO` script, which generates the questions and answers as `.parquet` files.

In [ ]:
!anyscale job submit -f configs/generate_questions_job.yaml

In [ ]:
QA_FOLDER = f"{os.environ['ANYSCALE_ARTIFACT_STORAGE']}/preference_tuning_summarization_example/qa_annotations_full_test/"
qa_ds = ray.data.read_parquet(QA_FOLDER)
example_rows = qa_ds.take(3)

In [11]:
for row in example_rows:
    print("TEXT:")
    print(row["text"])
    print()
    print("QUESTIONS:")
    print(row["qa_generation_questions"])
    print()
    print("ANSWERS:")
    print(list(row["qa_generation_answers"]))
    print("=" * 100)

TEXT:
By . Ashley Collman . PUBLISHED: . 08:43 EST, 12 July 2013 . | . UPDATED: . 09:20 EST, 12 July 2013 . Privileges reinstated: Dr Shakil Khan Afridi was allowed to see family last Wednesday after a 10-month ban on visitors following an interview he gave Fox News . The doctor who helped the CIA locate Osama Bin Laden is finally getting visits from his family at the Pakistani prison where he is being jailed. Dr Shakil Khan Afridi was convicted to 33 years in prison in May 2012 for 'acting against the state.' When the CIA was trying to confirm Bin Laden's presence in the Abbottabad compound, they sent Dr Afridi in under the auspices of giving out hepatitis B vaccinations. Investigators hoped the vaccinations would provide DNA evidence to confirm who was in the compound. While . Afridi was unsuccessful with the vaccinations, he was able to provide . enough details for the CIA to go through with their raid. When . the Pakistani government found out that one of their own was being used .

### Generate Summaries + Scores

Next, we will generate 10 summaries for each article in the training set and score them with our Q&A judging setup. 

The following command will run the `TODO` script, which takes in the folder of questions and generates the results to a new folder of `.parquet` files.

In [ ]:
!anyscale job submit -f `TODO`

In [17]:
SUMMARY_FOLDER = f"{os.environ['ANYSCALE_ARTIFACT_STORAGE']}/preference_tuning_summarization_example/summary_train_generation_mistralai_Mistral-7B-Instruct-v0.1_temp_0.8_judge_meta-llama_Meta-Llama-3.1-70B-Instruct/"
summary_ds = ray.data.read_parquet(SUMMARY_FOLDER)
example_rows = summary_ds.take(3)

Parquet Files Sample 0:   0%|          | 0/2 [00:00<?, ? file/s]

2024-08-13 01:45:40,404	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-08-13_00-32-31_998940_2689/logs/ray-data
2024-08-13 01:45:40,405	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet] -> LimitOperator[limit=3]


- ReadParquet->SplitBlocks(3) 1: 0 bundle [00:00, ? bundle/s]

- limit=3 2: 0 bundle [00:00, ? bundle/s]

Running 0: 0 bundle [00:00, ? bundle/s]

In [20]:
for row in example_rows:
    print("TEXT:")
    print(row["text"])
    print()
    print("QUESTIONS:")
    print(row["qa_generation_questions"])
    print()
    print("SUMMARY:")
    print(row["summary_generation_raw_model_output"])
    print()
    print("GROUND TRUTH ANSWERS:")
    print(list(row["qa_generation_answers"]))
    print()
    print("SUMMARY ANSWERS:")
    print(list(row["judge_mc_answers"]))
    print("=" * 100)

TEXT:
By . Riath Al-Samarrai . Follow @@riathalsam . Sergiy Stakhovsky has done it again. A year after dumping Roger Federer out of the second round at Wimbledon he sent Ernests Gulbis packing at the same stage. The world No 86 perhaps thought a posting on court 12 might keep him below the radar. But matches involving Gulbis are rarely quiet. And upsetting the 12th seed ensures he will inflate his reputation so long as he keeps his footing on grass. This was a performance of domination, a sustained beating that rarely, if ever, looked like letting up. He ultimately saw off the Latvian 6-4, 6-3, 7-6. Winner: Sergiy Stakhovsky of Ukraine is through to the third round of Wimbledon after beating Ernests Gulbis . Defeated: The Latvian reached the semi-finals of the French Open two weeks ago . It was impressive, but also a combustion of sorts from Gulbis, a man as volatile as he is gifted. The key moment came in the second set when, having already lost the first, Gulbis staved off a host of 

### Generate Preference Tuning Data

Next, we will generate 10 summaries for each article in the training set and score them with our Q&A judging setup. 

The following command will run the `TODO` script, which takes in the folder of summaries and outputs `.jsonl` files for training and validation.

In [ ]:
VALID_FILE = f"{os.environ['ANYSCALE_ARTIFACT_STORAGE']}/preference_tuning_summarization_example/training_data_summary_train_generation_mistralai_Mistral-7B-Instruct-v0.1_temp_0.8_judge_meta-llama_Meta-Llama-3.1-70B-Instruct/valid_geq_3_acc.jsonl/"
valid_ds = ray.data.read_json(VALID_FILE)
example_rows = valid_ds.take(3)

(autoscaler +1h20m24s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +1h20m24s) Cluster is recovering (reason: failed to maintain minimum healthy node requirement).
(autoscaler +1h20m26s) [autoscaler] [8xH100-80GB:192CPU-2048GB] Upscaling 4 node(s).
(autoscaler +1h20m26s) [autoscaler] [8xH100-80GB:192CPU-2048GB|lambda-oci-h100-80g-x8] Allocated machine lambda-oci-tmint to cluster.
(autoscaler +1h20m26s) [autoscaler] [8xH100-80GB:192CPU-2048GB|lambda-oci-h100-80g-x8] Allocated machine lambda-oci-1qevg to cluster.
(autoscaler +1h20m26s) [autoscaler] [8xH100-80GB:192CPU-2048GB|lambda-oci-h100-80g-x8] Allocated machine lambda-oci-tk22k to cluster.
(autoscaler +1h20m26s) [autoscaler] [8xH100-80GB:192CPU-2048GB|lambda-oci-h100-80g-x8] Allocated machine lambda-oci-1skwg to cluster.
(autoscaler +1h20m26s) [autoscaler] [8xH100-80GB:192CPU-2048GB|lambda-oci-h100-80g-x8] [us-east-1a] [on-demand] Launched 4 instances.
(aut

In [6]:
for row in example_rows:
    print("PROMPT:")
    print(row['chosen'][0]['content'])

PROMPT:
Given the following text, create a very short summary that is at most 2 sentences.

Text:
By . Tamara Cohen, Political Reporter . PUBLISHED: . 18:32 EST, 27 January 2013 . | . UPDATED: . 08:48 EST, 28 January 2013 . Deputy Prime Minister Nick Clegg and his wife Miriam are determined to keep the education of their 11-year-old son 'out of politics' Nick Clegg yesterday defended the possibility he may send his children to private schools as it emerged he and his wife Miriam have not even visited their local state school. He said the education of his 11-year-old son Antonio, who starts secondary school this year, should not be used as 'a political football' and that the couple would do 'what's best' for their children although he was braced for criticism. Last week the Liberal Democrat leader told listeners to his radio show he would send his son to a private school if he failed to find a place in a good comprehensive, saying he would use the state system 'if it works out', but tha

# Step 2: Fine-tuning

Now that we have the pre-processed dataset, we are ready to fine-tune `Mistral-7B-Instruct-v0.1` using DPO. On Anyscale, we've created an easy-to-use interface to do preference-tuning using `DPO`. We leverage Ray to overlap reference model log-probability calculation with model training to improve GPU utilization. Most implementations compute log probabilities synchronously with model training,

![hf model](assets/hf_dpo.png)

While our implementation using Ray is asynchronous:  


![assistant model](assets/anyscale_dpo.png)

Further, our use of Ray Data also implies that the compute configuration for the reference model can be completely decoupled with the policy model. For example, reference model calculation can run on a different node with zero code changes needed. 


To get started with DPO training, we provide the config for DPO in [configs/mistral_dpo_summarization.yaml](configs/mistral_dpo_summarization.yaml) . 


TODO: The provided config uses 6 and 2 A10s and doesn't utilize GPUs properly. We should improve logprob processor

In [10]:
!cat configs/mistral_dpo_summarization.yaml

model_id: mistralai/Mistral-7B-Instruct-v0.1
# Example summarization dataset with 10k examples for training with an average of 2.2k tokens per sample
train_path: s3://air-example-data/preference-tuning-summarization/train.jsonl
valid_path: s3://air-example-data/preference-tuning-summarization/valid.jsonl
task: "preference_tuning"
context_length: 4096
# For DPO, it is recommended to set a high `num_data_blocks_per_device` to not bottleneck the logp processor.
# We recommend not going beyond 20 so as to not spawn too many Ray actors. 
num_data_blocks_per_device: 16
num_devices: 6 # <--- runs training on 6 GPUs
train_batch_size_per_device: 2
eval_batch_size_per_device: 2
learning_rate: 5e-6
num_epochs: 3
no_gradient_checkpoint: False
output_dir: /mnt/local_storage/
deepspeed:
  config_path: deepspeed_configs/zero_3.json
worker_resources:
  accelerator_type:A10G: 1
flash_attention_2: True
padding: "longest"
preference_tuning_config:
  beta: 0.01
  logprob_processor_scaling_config:
    cust

In [ ]:
!llmforge anyscale finetune end-to-end-examples/fine-tune-preference/configs/mistral_dpo_summarization.yaml

# Step 3: Evaluation

Let's evaluate our trained model. Here we'll use two baselines: (1) the base model before finetuning (reference model in DPO) and (2) GPT-4o.

## Evaluation strategy

Our evaluation strategy involves the same Q&A scoring system as used while generating the preference data.



\<TODO: Add a nice diagram similar to data preprocessing, but just for the evaluation flow \>


\<TODO: Add description\>



# Step 4: Iterative-DPO (optional)

TODO